In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial

from timeit import default_timer
from utilities3 import *

from Adam import Adam

torch.manual_seed(0)
np.random.seed(0)

In [2]:
TRAIN_PATH = 'darcy/piececonst_r421_N1024_smooth1.mat'
TEST_PATH = 'darcy/piececonst_r421_N1024_smooth2.mat'

ntrain = 1000
ntest = 100

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

modes = 12
width = 32

r = 5
h = int(((421 - 1)/r) + 1)
s = h

In [3]:
reader = MatReader(TRAIN_PATH)
x_train = reader.read_field('coeff')[:ntrain,::r,::r][:,:s,:s]
y_train = reader.read_field('sol')[:ntrain,::r,::r][:,:s,:s]

reader.load_file(TEST_PATH)
x_test = reader.read_field('coeff')[:ntest,::r,::r][:,:s,:s]
y_test = reader.read_field('sol')[:ntest,::r,::r][:,:s,:s]

x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_test = x_normalizer.encode(x_test)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

x_train = x_train.reshape(ntrain,s,s,1)
x_test = x_test.reshape(ntest,s,s,1)

In [7]:
x_train[0,:10,:10,0]

tensor([[0.9995, 0.9836, 0.9777, 0.9935, 0.9896, 0.9856, 1.0035, 1.0035, 0.9915,
         0.9915],
        [0.9836, 0.9797, 0.9777, 0.9896, 0.9817, 0.9896, 0.9935, 0.9975, 0.9896,
         0.9975],
        [0.9777, 0.9719, 0.9738, 0.9797, 0.9817, 0.9797, 0.9915, 0.9876, 0.9797,
         0.9935],
        [0.9603, 0.9699, 0.9699, 0.9719, 0.9817, 0.9817, 0.9955, 0.9856, 0.9836,
         0.9836],
        [0.9641, 0.9680, 0.9699, 0.9797, 0.9836, 0.9836, 0.9856, 0.9836, 0.9817,
         0.9836],
        [0.9622, 0.9661, 0.9699, 0.9876, 0.9876, 0.9817, 0.9777, 0.9856, 0.9856,
         0.9817],
        [0.9896, 0.9876, 0.9856, 0.9836, 0.9817, 0.9797, 0.9817, 0.9836, 0.9856,
         0.9661],
        [0.9836, 0.9777, 0.9817, 0.9817, 0.9758, 0.9699, 0.9680, 0.9758, 0.9856,
         0.9836],
        [0.9719, 0.9699, 0.9719, 0.9622, 0.9603, 0.9622, 0.9622, 0.9699, 0.9797,
         0.9836],
        [0.9680, 0.9699, 0.9661, 0.9641, 0.9661, 0.9584, 0.9641, 0.9777, 0.9856,
         0.9915]])